In [1]:
import numpy as np
import torch
import torch.nn as nn
from IPython.display import display, HTML
from transformers import DistilBertModel, DistilBertTokenizer,logging
import matplotlib
import matplotlib.pyplot as plt
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
from captum.attr import Occlusion
import time
import pickle
from tqdm import tqdm
import spacy
from spacy import displacy
import seaborn as sns
import pandas as pd
import numpy as np
import collections

nlp = spacy.load('en_core_web_trf')
logging.set_verbosity_error()
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load model and set seed

In [3]:
# Bert mode
bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

class BERT(nn.Module):
    def __init__(self, bert):
        
        super(BERT, self).__init__()
        
        # Distil Bert model
        self.bert = bert
        ## Additional layers
        # Dropout layer
        self.dropout = nn.Dropout(0.3)
        # Relu 
        self.relu =  nn.ReLU()
        # Linear I 
        self.fc1 = nn.Linear(768, 512)
        # Linear II (Out)
        #self.fc2 = nn.Linear(512, 170)
        self.fc2 = nn.Linear(512, 817)

        
        # Softmax
        self.softmax = nn.LogSoftmax(dim=1)


    # Forward pass
    def forward(self, **kwargs):

        # Pass data trough bert and extract 
        cls_hs = self.bert(**kwargs)
        # Extract hidden state
        hidden_state = cls_hs.last_hidden_state
        # Only first is needed for classification
        pooler = hidden_state[:, 0]
        
        # Dense layer 1        
        x = self.fc1(pooler)
        # ReLU activation
        x = self.relu(x)
        # Drop out
        x = self.dropout(x)
        # Dense layer 2
        x = self.fc2(x)
        # Activation
        x = self.softmax(x)

        return x
    
# Load the entire model
model = BERT(bert)

# Load trained model (colab)
try:
    try:
        model_save_name = 'saved_weights_PartialDescriptions817.pt'
        path = F"/content/gdrive/My Drive/{model_save_name}"
        model.load_state_dict(torch.load(path))
        print('Google Success')

    except:
        model_save_name = 'saved_weights_PartialDescriptions817.pt'
        path = "../models/" + model_save_name
        model.load_state_dict(torch.load(path, 
                                         map_location=torch.device('cpu')))
        print('Local Success')
except:
    print('No pretrained model found.')
    
model.to(device)
model.eval()
model.zero_grad()

Local Success


In [4]:
# Set seeds
torch.manual_seed(333)
np.random.seed(333)

### Function and tokens

In [5]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [6]:
# Modify the prediction output and define a custom forward
def predict(inputs):
    return model(input_ids=inputs, attention_mask=attention_mask)[0]
def custom_forward(inputs):
    preds = predict(inputs)
    return torch.exp(preds)

In [7]:
# Tokenize functions
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [8]:
# Summarize and vis functions
def summarize_attributions_ig(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

def summarize_attributions_occ(attributions):
    return attributions.sum(axis=0)

def token_to_words(attribution, tokens):
    
    words = []
    attributes = []

    for attribute, word in zip(attribution, tokens):

        attribute = attribute.cpu().detach().numpy()
        if word == '[CLS]' or word == '[SEP]':
            words.append(word)
            attributes.append([attribute])
        elif not word.startswith('##'):
            words.append(word)
            attributes.append([attribute])
        elif word.startswith('##'):
            words[-1] += word.strip('##')
            attributes[-1] = np.append(attributes[-1], attribute)

    attribution = [np.sum(mean) for mean in attributes]
    return attribution, words

def colorize(attribution, tokens):
    
    template = """  
    <mark class="entity" style="
    background: {}; 
    padding: 0.4em 0.0em; 
    margin: 0.0em; 
    line-height: 2; 
    border-radius: 0.0em;
    ">{}<span style="
    font-size: 0.8em; 
    font-weight: bold; 
    line-height: 1; 
    border-radius: 0.0em;   
    text-align-last:center;
    vertical-align: middle;
    margin-left: 0rem;
    "></span></mark>
    """

    colored_string = ''
    normalized_and_mapped = matplotlib.cm.ScalarMappable(cmap=matplotlib.cm.PiYG).to_rgba(attribution)
    for idx, (word, color) in enumerate(zip(tokens, normalized_and_mapped)):
        
        word = word + ' '
        color = matplotlib.colors.rgb2hex(color[:3])
        if word.strip() == '[CLS]' or word.strip() == '[SEP]':
            color = '#ffffff'
        #print(color)
        colored_string += template.format(color, word)

    return colored_string

In [9]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)
occ = Occlusion(custom_forward)

## Single Example

In [11]:
text = 'The plant has a purple flower with white petals.'

In [12]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [13]:
attributions_ig = lig.attribute(inputs=input_ids,
                                baselines=ref_input_ids,
                                n_steps=20,
                                internal_batch_size=1,
                                return_convergence_delta=False)

attribution_occ = occ.attribute(inputs=input_ids, 
                                sliding_window_shapes=tuple([1,]), 
                                strides=None, 
                                baselines=None, 
                                target=None, 
                                additional_forward_args=None, 
                                perturbations_per_eval=1, 
                                show_progress=False)

In [14]:
# Sum the matrices
attributions_ig_sum = summarize_attributions_ig(attributions_ig)
attributions_occ_sum = summarize_attributions_occ(attribution_occ)

In [15]:
# Token to words
# Words are the same in this case
attributions_ig_words, words = token_to_words(attributions_ig_sum, all_tokens)
attributions_occ_words, words = token_to_words(attributions_occ_sum, all_tokens)

In [16]:
# IG
string = colorize(attributions_ig_words, words)
display(HTML(string))
# OCC
string = colorize(attributions_occ_words, words)
display(HTML(string))

## Loop

In [17]:
def extract_PoS(text):
    
    global attention_mask
    
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)
    

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    
    
    attributions_ig = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=20,
                                    internal_batch_size=1,
                                    return_convergence_delta=False)

    attributions_occ = occ.attribute(inputs=input_ids, 
                                    sliding_window_shapes=tuple([1,]), 
                                    strides=None, 
                                    baselines=None, 
                                    target=None, 
                                    additional_forward_args=None, 
                                    perturbations_per_eval=1, 
                                    show_progress=False)
    
    # Sum the matrices
    attributions_ig_sum = summarize_attributions_ig(attributions_ig)
    attributions_occ_sum = summarize_attributions_occ(attributions_occ)
    
    attributions_ig_words, words = token_to_words(attributions_ig_sum, all_tokens)
    attributions_occ_words, words = token_to_words(attributions_occ_sum, all_tokens)
    
    # NLP (Skip tokens)
    doc = nlp(' '.join(words[1:-1]))

    PoS = []
    
    #print(doc)
    
    for token, ig_att, occ_att in zip(doc, attributions_ig_words[1:-1], attributions_occ_words[1:-1]):
        #print(f'{token.text} -- {token.pos_} -- {ig_att} -- {occ_att}')
        
        PoS.append((token.lemma_, token.pos_, ig_att, occ_att))
        # Totals for visualization
        #PoS_totals[token.pos_].append(attribution)
    
    return PoS
    # Sort on attribution    
    #PoS.sort(key = lambda x : (-x[2]))

        #PoS_list['Sorghum arundinaceum'].append(PoS)

In [18]:
#extract_PoS(text)

In [19]:
text_list = ['The plant is purple.',
             'The species has a spiky stem and has spikets on the bark',
             'The stem is brownish with grey spikelets.',
             'The leaves are green with an orange glow at the tips.']

In [20]:
datadict = pickle.load(open('../data/description/04_TRAIN_0-817_PLANTS.pkl', 'rb'))
text_list = [data for data in datadict['Muhlenbergia ligularis']]

In [ ]:
Pos_list = [extract_PoS(text) for text in tqdm(text_list)]

  8%|███▎                                      | 12/153 [01:37<15:42,  6.69s/it]

In [ ]:
#Pos_list

## Vis

In [ ]:
flat_list = []
for idx, sublist in enumerate(Pos_list):
    for item in sublist:
        flat_list.append((idx, item[0], item[1], item[2], item[3]))

In [ ]:
df = pd.DataFrame(flat_list, columns=['sentence', 'word', 'pos', 'ig', 'occ'])

In [ ]:
# Scale
df['ig_scaled'] = (df['ig'] - df['ig'].mean()) / (df['ig'].max() - df['ig'].min())
df['occ_scaled'] = (df['occ'] - df['occ'].mean()) / (df['occ'].max() - df['occ'].min())

In [ ]:
idx = []
for sentence in df['sentence'].unique():
    idx += list(df[df['sentence'] == sentence]['ig_scaled'].sort_values(ascending=False)[0:3].index)
    #idx += list(df[(df['sentence'] == sentence) & (df['pos'] == 'NOUN') | (df['pos'] == 'ADJ')]['ig_scaled'].sort_values(ascending=False)[0:3].index)
df_highest = df.iloc[idx]

df_highest_melt = df_highest.melt(id_vars=['pos'], value_vars=['ig_scaled', 'occ_scaled'])

In [ ]:
#df_highest.groupby('pos').count()

In [ ]:
df_melt = df.melt(id_vars=['pos'], value_vars=['ig_scaled', 'occ_scaled'])

In [ ]:
with sns.axes_style("whitegrid"):
    f, ax = plt.subplots(figsize=(25, 16))
    ax = sns.violinplot(x="pos", y="value", hue="variable",
                        data=df_melt, 
                        palette="muted", 
                        split=False,
                        #inner="quartile",
                        bw=.2)

In [ ]:
# Sort the values
mean_values = [(column, float(df_melt[df_melt['pos'] == column].mean())) for column in df['pos'].unique()]
mean_values.sort(key = lambda x : (x[1]))

sorter = [pos_value for (pos_value, mean) in mean_values]
#print(sorter)

In [ ]:
df_melt_sorted = df_melt.set_index('pos')
df_melt_sorted = df_melt_sorted.loc[sorter]
df_melt_sorted['pos'] = df_melt_sorted.index

In [ ]:
with sns.axes_style("whitegrid"):
    f, ax = plt.subplots(figsize=(15, 8))
    # Initialize the figure
    #f, ax = plt.subplots()
    sns.despine(bottom=True, left=True)

    # Show each observation with a scatterplot
    sns.stripplot(x="value", y="pos", hue="variable",
                  data=df_melt, dodge=True, alpha=.25, zorder=1)

    # Show the conditional means, aligning each pointplot in the
    # center of the strips by adjusting the width allotted to each
    # category (.8 by default) by the number of hue levels
    sns.pointplot(x="value", y="pos", hue="variable",
                  data=df_melt, dodge=.4,
                  join=False, palette="dark",
                  markers="d", scale=.75, ci=None)

    # Improve the legend
    #handles, labels = ax.get_legend_handles_labels()
    #ax.legend(handles[3:], labels[3:], title="gradients",
    #          handletextpad=0, columnspacing=1,
    #          loc="lower right", ncol=3, frameon=True)

In [ ]:
# Sort the values
mean_values = [(column, float(df_highest_melt[df_highest_melt['pos'] == column].mean())) for column in df_highest['pos'].unique()]
mean_values.sort(key = lambda x : (x[1]))

sorter = [pos_value for (pos_value, mean) in mean_values]

df_melt_sorted = df_highest_melt.set_index('pos')
df_melt_sorted = df_melt_sorted.loc[sorter]
df_melt_sorted['pos'] = df_melt_sorted.index

In [ ]:
with sns.axes_style("whitegrid"):
    f, ax = plt.subplots(figsize=(25, 16))
    ax = sns.violinplot(x="pos", y="value", hue="variable",
                        data=df_highest_melt, 
                        palette="muted", 
                        split=True,
                        #inner="quartile",
                        bw=.2)

# Stats

In [ ]:
from scipy import stats 

In [ ]:
a = df[df['pos'] == 'PROPN']['occ_scaled']

In [ ]:
stats.ttest_1samp(a, 0, nan_policy='propagate', alternative='two-sided')

In [ ]:
a.mean()